In [1]:
import pandas as pd
import altair as alt
from altair_saver import save
import chromedriver_autoinstaller as ca
ca.install()

'/home/sengi/.local/lib/python3.10/site-packages/chromedriver_autoinstaller/112/chromedriver'

In [2]:
file_path = "./data.csv"
df = pd.read_csv(file_path)

In [3]:
df.head()


,COU,Country,POL,Pollutant,VAR,Variable,YEA,Year,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
0,AUS,Australia,GHG,Greenhouse gases,TOTAL,Total emissions excluding LULUCF,1990,1990,T_CO2_EQVT,Tonnes of CO2 equivalent,3,Thousands,NaN,NaN,425624.307,NaN,NaN
1,AUS,Australia,GHG,Greenhouse gases,TOTAL,Total emissions excluding LULUCF,1991,1991,T_CO2_EQVT,Tonnes of CO2 equivalent,3,Thousands,NaN,NaN,425686.445,NaN,NaN
2,AUS,Australia,GHG,Greenhouse gases,TOTAL,Total emissions excluding LULUCF,1992,1992,T_CO2_EQVT,Tonnes of CO2 equivalent,3,Thousands,NaN,NaN,429473.085,NaN,NaN
3,AUS,Australia,GHG,Greenhouse gases,TOTAL,Total emissions excluding LULUCF,1993,1993,T_CO2_EQVT,Tonnes of CO2 equivalent,3,Thousands,NaN,NaN,430381.731,NaN,NaN
4,AUS,Australia,GHG,Greenhouse gases,TOTAL,Total emissions excluding LULUCF,1994,1994,T_CO2_EQVT,Tonnes of CO2 equivalent,3,Thousands,NaN,NaN,430848.498,NaN,NaN


In [4]:
df['Pollutant'].unique()

array(['Greenhouse gases', 'Carbon dioxide', 'Methane', 'Nitrous oxide',
       'Hydrofluorocarbons', 'Perfluorocarbons', 'Sulphur hexafluoride',
       'Nitrogen trifluoride', 'Unspecified mix of HFCs and PFCs'],
      dtype=object)

In [5]:
tmp = df[df["Country"].str.contains("China")]
tmp["Year"].unique()
len(df["COU"].unique())

63

In [6]:
# get the most recent year with data from most number of countries
n_country = []
max_year = 0
max_country = 0
for i in df["Year"].unique():
    tmp_df = df[(df['Year'] == i) & (df['Unit'] == "Tonnes of CO2 equivalent") \
                 &(~df['COU'].str.contains("OECD")) & (df["COU"] != "EU27_2020")]
    cur = len(tmp_df["COU"].unique())
    n_country.append(cur)
    max_year = max_year if cur<max_country else i
    max_country = max(max_country, cur)
print(max_year)

2010


In [7]:
# Filter the data for 2010 (most recent year with data from most number of countries) 
# and group by country, summing the emissions

df_data = df[(df['Year'] == max_year) & (df['Unit'] == "Tonnes of CO2 equivalent") \
             & (~df['COU'].str.contains("OECD")) & (df["COU"] != "EU27_2020") \
             & (df['Pollutant'] == 'Greenhouse gases')\
             & (df['VAR'] == 'TOTAL')]
# grouped_data = df_data.groupby('Country')['Value'].sum().reset_index()

# Sort the data and select the top 10 countries by emissions
top_10_countries = df_data.nlargest(10, 'Value')

# Create the Altair chart
earnest_chart = alt.Chart(top_10_countries).mark_bar().encode(
    alt.X('Value:Q', axis=alt.Axis(title="Thousand Tonnes of CO2 equivalent")),
    alt.Y('Country:N', sort='-x', axis=alt.Axis(title='Country')),
#     tooltip=['Country', 'Total_Emissions']
).properties(
    title=f'Top 10 Greenhouse Gas Emitting Countries in {max_year}',
    width=600,
    height=400
)

# Display the chart
# earnest_chart.show()
save(earnest_chart, "a2_earnest.png")


In [8]:
# Not all countries have data for each year, so we need to align the data. 
import pandas as pd

df = df[(~df['COU'].str.contains("OECD")) & (df["COU"] != "EU27_2020") & (df['VAR'] == 'TOTAL')]
# Group the data by 'Country' and count the unique years for each country
country_year_counts = df.groupby('COU')['Year'].nunique().reset_index(name='Unique_Years')

# Calculate the total number of unique years in the dataset
total_years = df['Year'].nunique()

# Filter the countries that have data for all years
countries_with_complete_data = country_year_counts[country_year_counts['Unique_Years'] == total_years]['COU']

# Filter the data for the years between 2013 and 2014 and group by year, summing the emissions
time_filtered_data = df[(df['Year'] >= 2017) & (df['Year'] <= 2018) \
                         & (df['Unit'] == "Tonnes of CO2 equivalent") \
                         & (df['COU'].isin(countries_with_complete_data))
                         & (df['Pollutant'] == 'Greenhouse gases') \
                         & (df['PowerCode'] == "Thousands")]
grouped_data = time_filtered_data.groupby('Year')['Value'].sum().reset_index()

# Create the deceptive Altair chart with a misleading y-axis scale
deceptive_chart = alt.Chart(grouped_data).mark_bar().encode(
    alt.X('Year:N', axis=alt.Axis(title='Year')),
    alt.Y('Value:Q',
          scale=alt.Scale(domain=[1.69e7, 1.72e7]),  # Set a deceptive scale for the y-axis
          axis=alt.Axis(title="Thousand Tonnes of CO2 equivalent")),
    tooltip=['Year', 'Value']
).properties(
    title='Global Greenhouse Gas Emissions: 2017-2018',
    width=600,
    height=400
)

# Display the chart
# deceptive_chart.show()
deceptive_chart.save('a2_deceptive.png', fmt='png')


In [9]:
for i in time_filtered_data["Year"].unique():
    tmp = time_filtered_data[time_filtered_data['Year']==i]
    print(len(tmp["COU"]))

43
43


In [10]:
tmp = time_filtered_data[time_filtered_data['Year']==2017]
set2017 = set(tmp['Country'].unique())
tmp = time_filtered_data[time_filtered_data['Year']==2020]
set2020 = set(tmp['Country'].unique())
diff = set2017.difference(set2020)
print(list(diff))

['Latvia', 'Monaco', 'Bulgaria', 'Kazakhstan', 'Croatia', 'Poland', 'Sweden', 'Lithuania', 'Malta', 'Japan', 'Liechtenstein', 'Russia', 'Luxembourg', 'New Zealand', 'Netherlands', 'Hungary', 'Ukraine', 'Switzerland', 'Denmark', 'Iceland', 'Czech Republic', 'Romania', 'Slovak Republic', 'Norway', 'United States', 'Slovenia', 'Finland', 'Portugal', 'Türkiye', 'United Kingdom', 'France', 'Austria', 'Greece', 'Ireland', 'Spain', 'Cyprus', 'Canada', 'Belgium', 'Italy', 'Australia', 'Estonia', 'Belarus', 'Germany']


In [11]:
tmp = df[df['COU']=='CRI']
tmp[tmp['Year']==2020]

,COU,Country,POL,Pollutant,VAR,Variable,YEA,Year,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
